In [1]:
# ---- load globals ----
source("R/globals.R", local = TRUE)

source("R/data_loading.R", local = TRUE)


source("R/module_rotate.R", local = TRUE)

library(plotly)       # ← 新增
library(base64enc)    # ← 新增（如果保留 data URI 路线）
source("R/module_select.R") 



Attaching package: ‘jsonlite’


The following object is masked from ‘package:shiny’:

    validate



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout




In [3]:
# # ---- Shiny UI ----
# ui <- fluidPage(

#   fileInput("matrix_file",   "上传 matrix.mtx.gz"),
#   fileInput("barcodes_file", "上传 barcodes.tsv.gz"),
#   fileInput("features_file", "上传 features.tsv.gz)"),
#   fileInput("positions_file","上传 tissue_positions_list.csv"),
#   fileInput("scalef_file",   "上传 scalefactors_json.json"),
#   fileInput("image_file",    "上传 HE 图像 (png/jpg)"),


#   titlePanel("Visium HE overlay + gene expression (Lv0)"),
#   sidebarLayout(
#     sidebarPanel(
#       selectInput("gene", "选择基因：", choices = gene_choices, selected = gene_choices[1]),
#       checkboxInput("log1p", "log1p 变换", value = TRUE),
#       sliderInput("ptsize", "点大小", min = 1, max = 6, value = 3, step = 0.5),
#       sliderInput("alpha", "点透明度", min = 0.1, max = 1, value = 0.9, step = 0.1),
#       helpText("温馨提示：只加载了前 ", MAX_GENES, " 个基因作为示例；后续可做基因搜索。")
#     ),
#     mainPanel(
#       plotOutput("he_plot", height = "750px", width = "100%"),
#       tags$hr(),
#       verbatimTextOutput("dbg")   # 调试输出

#     )
#   )
# )

ui <- fluidPage(
  titlePanel("Visium HE overlay + gene expression (Lv0)"),
  sidebarLayout(
    sidebarPanel(
      # 上传控件
      fileInput("matrix_file",   "上传 matrix.mtx(.gz)"),
      fileInput("barcodes_file", "上传 barcodes.tsv(.gz)"),
      fileInput("features_file", "上传 features.tsv(.gz)"),
      fileInput("positions_file","上传 tissue_positions_list.csv"),
      fileInput("scalef_file",   "上传 scalefactors_json.json"),
      fileInput("image_file",    "上传 HE 图像 (png/jpg)"),

      # 基因选择：初始为空，加载后用 server 端 updateSelectInput() 填
      selectInput("gene", "选择基因：", choices = character(0), selected = NULL),


      checkboxInput("log1p", "log1p 变换", value = TRUE),
      sliderInput("ptsize", "点大小", min = 1, max = 6, value = 3, step = 0.5),
      sliderInput("alpha", "点透明度", min = 0.1, max = 1, value = 0.9, step = 0.1),
      helpText("请先上传 10x 矩阵三件套（matrix/barcodes/features）再选择基因。"),

      tags$hr(),
      rotateImageUI("rot")
    ),


    # mainPanel(
    #   plotOutput("he_plot", height = "750px", width = "100%"),
    #   tags$hr(),
    #   verbatimTextOutput("dbg")
    # )
    mainPanel(
        fluidRow(
          column(
            width = 6,
            plotOutput("he_plot", height = "750px", width = "100%")
          ),
          column(
            width = 6,
            selectSpotsUI("sel")          # ← 新增：选择模块 UI
          )
        ),
        hr(),
        verbatimTextOutput("dbg")
      )
    )
  )


In [4]:
# ---- 读图：确保返回 raster ----
load_image <- function(path) {
  if (grepl("\\.png$", path, ignore.case = TRUE)) {
    arr <- png::readPNG(path)     # H x W x C, 0..1
  } else if (grepl("\\.jpe?g$", path, ignore.case = TRUE)) {
    arr <- jpeg::readJPEG(path)
  } else stop("Only PNG/JPG supported.")

  # 灰度图转 3 通道
  if (length(dim(arr)) == 2L) {
    arr <- array(rep(arr, each = 3), dim = c(dim(arr), 3))
  }
  img_h <- dim(arr)[1]; img_w <- dim(arr)[2]
  img_rs <- as.raster(arr)        # 关键：转 raster
  list(img = img_rs, img_w = img_w, img_h = img_h)
}

server <- function(input, output, session) {
  rv <- reactiveValues(
    counts=NULL, gene_names=NULL, barcodes=NULL,
    pos=NULL, hires_scale=NULL, img=NULL, img_w=NULL, img_h=NULL, img_path=NULL
  )

  # ---- 加载 counts/features/barcodes ----
  observeEvent(list(input$matrix_file, input$barcodes_file, input$features_file), {
    req(input$matrix_file, input$barcodes_file, input$features_file)
    paths <- list(input$matrix_file$datapath, input$barcodes_file$datapath, input$features_file$datapath)
    message("Loading counts ...")
    res <- load_counts(paths[[1]], paths[[2]], paths[[3]])
    rv$counts     <- res$counts
    rv$gene_names <- res$gene_names
    rv$barcodes   <- res$barcodes
    updateSelectInput(session, "gene", choices = head(rv$gene_names, MAX_GENES),
                      selected = head(rv$gene_names, 1))
  })

  # ---- 加载 positions ----
  observeEvent(input$positions_file, {
    req(input$positions_file)
    message("Loading positions ...")
    pos <- load_positions(input$positions_file$datapath)
    if (!is.null(rv$counts)) {
      pos <- subset(pos, in_tissue == 1 & barcode %in% colnames(rv$counts))
    } else {
      pos <- subset(pos, in_tissue == 1)
    }
    rv$pos <- pos
  })

  # ---- 加载 scalefactor ----
  observeEvent(input$scalef_file, {
    req(input$scalef_file)
    message("Loading scalefactor ...")
    rv$hires_scale <- load_scalef(input$scalef_file$datapath)
    if (!is.null(rv$pos)) rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
  })

  # ---- 加载图像 ----
  observeEvent(input$image_file, {
    req(input$image_file)
    message("Loading image ...")
    im <- load_image(input$image_file$datapath)
    rv$img   <- im$img      # raster
    rv$img_w <- im$img_w
    rv$img_h <- im$img_h
    rv$img_path <- input$image_file$datapath
  })

  # ---- pos 更新后补映射 ----
  observeEvent(rv$pos, {
    if (!is.null(rv$pos) && !is.null(rv$hires_scale) && is.null(rv$pos$x_hires)) {
      rv$pos <- to_hires_coords(rv$pos, rv$hires_scale)
    }
  })

  # ---- 表达向量 ----
  expr_vec <- reactive({
    req(input$gene, rv$counts, rv$pos)
    v <- if (input$gene %in% rownames(rv$counts)) {
      as.numeric(rv$counts[input$gene, rv$pos$barcode])
    } else rep(0, nrow(rv$pos))
    if (isTRUE(input$log1p)) v <- log1p(v)
    v
  })

  rot <- rotateImageServer("rot", image_path_reactive = reactive(rv$img_path))

  current_img_obj <- reactive({
    # 优先用旋转后的；否则用原图
    if (!is.null(rot$get())) {
      rot$get()
    } else if (!is.null(rv$img)) {
      list(img = rv$img, img_w = rv$img_w, img_h = rv$img_h)
    } else {
      NULL
    }
  })

sel <- selectSpotsServer(
    "sel",
    pos_reactive  = reactive(rv$pos),      # 需要包含 barcode/x_hires/y_hires
    expr_reactive = expr_vec,              # 你已有的表达向量 reactive
    img_reactive  = current_img_obj        # 背景图（旋转后/原图皆可）
  )


  # ---- 画图 ----
  output$he_plot <- renderPlot({
    req(rv$pos)

    # 优先使用旋转后的背景；若还没有，就用原始背景
    img_obj <- NULL
    rot_obj <- tryCatch(rot$get(), error = function(e) NULL)
    if (!is.null(rot_obj)) {
      img_obj <- rot_obj
    } else if (!is.null(rv$img)) {
      img_obj <- list(img = rv$img, img_w = rv$img_w, img_h = rv$img_h)
    }
    req(img_obj$img, img_obj$img_w, img_obj$img_h)

    v  <- expr_vec()
    df <- data.frame(x = rv$pos$x_hires, y = rv$pos$y_hires, expr = v)

    ggplot() +
      annotation_raster(img_obj$img, xmin = -Inf, xmax = Inf, ymin = -Inf, ymax = Inf, interpolate = TRUE) +
      geom_point(data = df, aes(x, y, color = expr),
                size = input$ptsize, alpha = input$alpha) +
      scale_color_viridis_c(name = input$gene %||% "expression") +
      scale_y_reverse() +
      coord_fixed(xlim = c(0, img_obj$img_w), ylim = c(0, img_obj$img_h), expand = FALSE) +
      theme_void()
  }, res = 120)


  # ---- 调试信息（可留可去） ----
  output$dbg <- renderPrint({
    list(
      have_counts = !is.null(rv$counts),
      have_pos    = !is.null(rv$pos),
      have_scale  = !is.null(rv$hires_scale),
      have_img    = !is.null(rv$img),
      img_class   = if (!is.null(rv$img)) class(rv$img)
    )
  })
}


In [5]:
shinyApp(ui, server)


Listening on http://127.0.0.1:6133

Loading required namespace: magick

Warning message:
“The 'plotly_selected' event tied a source ID of '' is not registered. In order to obtain this event data, please add `event_register(p, 'plotly_selected')` to the plot (`p`) that you wish to obtain event data from.”
Loading counts ...

Loading positions ...

Loading scalefactor ...

Loading image ...

